# Importations

In [1]:
import numpy as np

# Activation functions classes

In [2]:
class sigmoid:
    def __init__(self):
        None
    
    def function(self, z):
        return 1. / (1. + np.exp(-z))
    
    def derivative(self, z):
        return self.function(z) * (1. - self.function(z))

class tanh:
    def __init__(self):
        None
    
    def function(self, z):
        return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))
    
    def derivative(self, z):
        return 1 - self.function ** 2
    
class relu:
    def __init__(self):
        None
    
    def function(self, z):
        return np.maximum(0., z)
    
    def derivative(self, z):
        if z < 0:
            res = 0
        else:
            res = 1
        return res

class leakyrelu:
    def __init__(self, beta):
        self.beta = beta
    
    def function(self, z):
        return np.maximum(self.beta * z, z)
    
    def derivative(self, z):
        if z < 0:
            res = self.beta
        else:
            res = 1
        return res

# Cost functions classes

In [3]:
class MeanSquaredCost:
    def __init__(self):
        None
        
    def compute(self, y_hat, y):
        return 0.5 * np.linalg.norm(y - y_hat) ** 2
        
    def final_derivative(self, y_hat, y):
        # returns dL/dy
        return y - y_hat

class CrossEntropyCost:
    def __init__(self):
        None
        
    def compute(self, y_hat, y):
        return np.mean(- y * np.log(y_hat) - (1 - y) * np.log(1 - y_hat))
        
    def final_derivative(self, y_hat, y):
        # returns dL/dy
        return y_hat - y

# Dense layer class

In [8]:
class Dense:
    
    def __init__(self, units, input_dim, activation='sigmoid', initialization='random'):        
        # Activation function
        if activation == 'sigmoid':
            self.activation = sigmoid()
        elif activation == 'tanh':
            self.activation = tanh()
        elif activation == 'relu':
            self.activation = relu()
        elif activation == 'leakyrelu':
            self.activation = leakyrelu(beta=0.01)
        
        # Layer parameters (zero initialization)
        if initialization == 'zero':
            self.W = np.zeros((units, input_dim))
            self.b = np.zeros((units, 1))
        elif initialization == 'random':
            self.W = np.random.randn(units, input_dim) * 0.01
            self.b = np.zeros((units, 1))
        
        # Cache
        self.input = None
        self.Z = None
        self.dW = None
        self.db = None
    
    def forward(self, A):
        # input : A[l-1], cache : Z[l], A[l-1], output : A[l]
        # Z[l] =  W[l] * A[l-1] + b[l]
        # A[l] = f(Z[l])
        
        # saved in cache
        self.input = A
        
        Z = np.dot(self.W, self.input) + self.b
        A = self.activation.function(Z)
        
        # saved in cache
        self.Z = Z
        
        return A
        
    def backward(self, dA=None, dZ=None):
        # input : dA[l] or dZ[l], cache : dW, db, output : dA[l-1]
        
        if dA is not None:
            dZ = self.activation.derivative(self.Z) *  dA
        
        m = self.input.shape[1]
        
        dW = (1 / m) * np.dot(dZ, self.input.T)
        db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
        dA = np.dot(self.W.T, dZ)
        
        # saved in cache
        self.dW = dW
        self.db = db
        
        return dA
                
        
    def update_weights(self, learning_rate):
        # W[l] = W[l] - alpha * dW[l]
        # b[l] = b[l] - alpha * db[l]
        
        self.W = self.W - learning_rate * self.dW
        self.b = self.b - learning_rate * self.db

# Neural network class

In [13]:
class NeuralNetwork:
    
    def __init__(self, layers):
        self.layers = layers
        self.nb_layers = len(layers)
    
    def train(self, X_train, y_train, nb_epochs, learning_rate, cost):
        for i in range(1, nb_epochs + 1):
            print('Step :', i, '/', nb_epochs)

            # Forward propagation
            print('Forward pass')
            A = X_train
            for l in range(self.nb_layers):
                A = self.layers[l].forward(A)
            out = A

            # Compute cost
            c = cost.compute(out, y_train)
            dZ = cost.final_derivative(out, y_train)
            print('Cost = ', c)

            # Backward propagation
            print('Backward pass')
            dZ = cost.final_derivative(out, y_train)
            dA = self.layers[self.nb_layers - 1].backward(dZ=dZ)
            for l in range(1, self.nb_layers):
                dA = self.layers[self.nb_layers - l - 1].backward(dA=dA)

            # Update weights
            print('Updating weights...')
            for l in range(self.nb_layers):
                self.layers[l].update_weights(learning_rate)
        
    def predict(self, X_test):
        A = X_test
        for l in range(self.nb_layers):
            A = self.layers[l].forward(A)
        return A

# Dataset

In [14]:
X1 = np.random.randn(100, 2) + 1
Y1 = np.zeros((100, 1))

X2 = np.random.randn(100, 2) - 1
Y2 = np.ones((100, 1))

X = np.concatenate((X1, X2))
y = np.concatenate((Y1, Y2))

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

X_train, X_test, y_train, y_test = X_train.T, X_test.T, y_train.T, y_test.T

print(np.shape(X_train))

(2, 160)


# Create model

In [15]:
layers = []
layers.append(Dense(units=3, input_dim=2, activation='sigmoid'))
layers.append(Dense(units=1, input_dim=3, activation='sigmoid'))

net = NeuralNetwork(layers)

# Train model

In [16]:
nb_epochs = 20000
learning_rate = 0.1
cost = CrossEntropyCost()

net.train(X_train, y_train, nb_epochs, learning_rate, cost)

Step : 1 / 20000
Forward pass
Cost =  0.6931143954525559
Backward pass
Updating weights...
Step : 2 / 20000
Forward pass
Cost =  0.6931034140413745
Backward pass
Updating weights...
Step : 3 / 20000
Forward pass
Cost =  0.6930931751420679
Backward pass
Updating weights...
Step : 4 / 20000
Forward pass
Cost =  0.6930836005042045
Backward pass
Updating weights...
Step : 5 / 20000
Forward pass
Cost =  0.693074618754918
Backward pass
Updating weights...
Step : 6 / 20000
Forward pass
Cost =  0.6930661647490602
Backward pass
Updating weights...
Step : 7 / 20000
Forward pass
Cost =  0.6930581789789418
Backward pass
Updating weights...
Step : 8 / 20000
Forward pass
Cost =  0.6930506070381405
Backward pass
Updating weights...
Step : 9 / 20000
Forward pass
Cost =  0.6930433991343634
Backward pass
Updating weights...
Step : 10 / 20000
Forward pass
Cost =  0.6930365096468154
Backward pass
Updating weights...
Step : 11 / 20000
Forward pass
Cost =  0.6930298967239358
Backward pass
Updating weights..

Updating weights...
Step : 215 / 20000
Forward pass
Cost =  0.615299741746879
Backward pass
Updating weights...
Step : 216 / 20000
Forward pass
Cost =  0.6135259989754145
Backward pass
Updating weights...
Step : 217 / 20000
Forward pass
Cost =  0.6117292968530744
Backward pass
Updating weights...
Step : 218 / 20000
Forward pass
Cost =  0.6099098575371629
Backward pass
Updating weights...
Step : 219 / 20000
Forward pass
Cost =  0.6080679148313576
Backward pass
Updating weights...
Step : 220 / 20000
Forward pass
Cost =  0.6062037139300058
Backward pass
Updating weights...
Step : 221 / 20000
Forward pass
Cost =  0.6043175111599538
Backward pass
Updating weights...
Step : 222 / 20000
Forward pass
Cost =  0.602409573720805
Backward pass
Updating weights...
Step : 223 / 20000
Forward pass
Cost =  0.600480179424401
Backward pass
Updating weights...
Step : 224 / 20000
Forward pass
Cost =  0.5985296164342169
Backward pass
Updating weights...
Step : 225 / 20000
Forward pass
Cost =  0.59655818300

Step : 544 / 20000
Forward pass
Cost =  0.16054045476348322
Backward pass
Updating weights...
Step : 545 / 20000
Forward pass
Cost =  0.16019933552505317
Backward pass
Updating weights...
Step : 546 / 20000
Forward pass
Cost =  0.15986058514763687
Backward pass
Updating weights...
Step : 547 / 20000
Forward pass
Cost =  0.15952418257224116
Backward pass
Updating weights...
Step : 548 / 20000
Forward pass
Cost =  0.15919010696140828
Backward pass
Updating weights...
Step : 549 / 20000
Forward pass
Cost =  0.15885833769666102
Backward pass
Updating weights...
Step : 550 / 20000
Forward pass
Cost =  0.15852885437597625
Backward pass
Updating weights...
Step : 551 / 20000
Forward pass
Cost =  0.15820163681128763
Backward pass
Updating weights...
Step : 552 / 20000
Forward pass
Cost =  0.15787666502601688
Backward pass
Updating weights...
Step : 553 / 20000
Forward pass
Cost =  0.1575539192526328
Backward pass
Updating weights...
Step : 554 / 20000
Forward pass
Cost =  0.15723337993023917
B

Updating weights...
Step : 830 / 20000
Forward pass
Cost =  0.11399111318587893
Backward pass
Updating weights...
Step : 831 / 20000
Forward pass
Cost =  0.11391986371826804
Backward pass
Updating weights...
Step : 832 / 20000
Forward pass
Cost =  0.11384892415116137
Backward pass
Updating weights...
Step : 833 / 20000
Forward pass
Cost =  0.11377829273238141
Backward pass
Updating weights...
Step : 834 / 20000
Forward pass
Cost =  0.11370796772196161
Backward pass
Updating weights...
Step : 835 / 20000
Forward pass
Cost =  0.11363794739204525
Backward pass
Updating weights...
Step : 836 / 20000
Forward pass
Cost =  0.11356823002678462
Backward pass
Updating weights...
Step : 837 / 20000
Forward pass
Cost =  0.11349881392224177
Backward pass
Updating weights...
Step : 838 / 20000
Forward pass
Cost =  0.11342969738628968
Backward pass
Updating weights...
Step : 839 / 20000
Forward pass
Cost =  0.11336087873851461
Backward pass
Updating weights...
Step : 840 / 20000
Forward pass
Cost =  

Updating weights...
Step : 1049 / 20000
Forward pass
Cost =  0.10359603914765522
Backward pass
Updating weights...
Step : 1050 / 20000
Forward pass
Cost =  0.1035652617032227
Backward pass
Updating weights...
Step : 1051 / 20000
Forward pass
Cost =  0.10353458816523946
Backward pass
Updating weights...
Step : 1052 / 20000
Forward pass
Cost =  0.10350401807533587
Backward pass
Updating weights...
Step : 1053 / 20000
Forward pass
Cost =  0.10347355097760498
Backward pass
Updating weights...
Step : 1054 / 20000
Forward pass
Cost =  0.10344318641858688
Backward pass
Updating weights...
Step : 1055 / 20000
Forward pass
Cost =  0.1034129239472529
Backward pass
Updating weights...
Step : 1056 / 20000
Forward pass
Cost =  0.10338276311499006
Backward pass
Updating weights...
Step : 1057 / 20000
Forward pass
Cost =  0.10335270347558585
Backward pass
Updating weights...
Step : 1058 / 20000
Forward pass
Cost =  0.10332274458521282
Backward pass
Updating weights...
Step : 1059 / 20000
Forward pass

...

Updating weights...
Step : 18859 / 20000
Forward pass
Cost =  0.08335242287012812
Backward pass
Updating weights...
Step : 18860 / 20000
Forward pass
Cost =  0.0833521704497785
Backward pass
Updating weights...
Step : 18861 / 20000
Forward pass
Cost =  0.08335191802223471
Backward pass
Updating weights...
Step : 18862 / 20000
Forward pass
Cost =  0.08335166558749418
Backward pass
Updating weights...
Step : 18863 / 20000
Forward pass
Cost =  0.08335141314555414
Backward pass
Updating weights...
Step : 18864 / 20000
Forward pass
Cost =  0.0833511606964119
Backward pass
Updating weights...
Step : 18865 / 20000
Forward pass
Cost =  0.0833509082400647
Backward pass
Updating weights...
Step : 18866 / 20000
Forward pass
Cost =  0.08335065577650994
Backward pass
Updating weights...
Step : 18867 / 20000
Forward pass
Cost =  0.08335040330574484
Backward pass
Updating weights...
Step : 18868 / 20000
Forward pass
Cost =  0.08335015082776671
Backward pass
Updating weights...
Step : 18869 / 20000
Fo

Updating weights...
Step : 19140 / 20000
Forward pass
Cost =  0.08328119993128766
Backward pass
Updating weights...
Step : 19141 / 20000
Forward pass
Cost =  0.08328094538431623
Backward pass
Updating weights...
Step : 19142 / 20000
Forward pass
Cost =  0.08328069082940509
Backward pass
Updating weights...
Step : 19143 / 20000
Forward pass
Cost =  0.08328043626655166
Backward pass
Updating weights...
Step : 19144 / 20000
Forward pass
Cost =  0.08328018169575335
Backward pass
Updating weights...
Step : 19145 / 20000
Forward pass
Cost =  0.08327992711700749
Backward pass
Updating weights...
Step : 19146 / 20000
Forward pass
Cost =  0.08327967253031157
Backward pass
Updating weights...
Step : 19147 / 20000
Forward pass
Cost =  0.08327941793566296
Backward pass
Updating weights...
Step : 19148 / 20000
Forward pass
Cost =  0.08327916333305904
Backward pass
Updating weights...
Step : 19149 / 20000
Forward pass
Cost =  0.08327890872249724
Backward pass
Updating weights...
Step : 19150 / 20000

Updating weights...
Step : 19375 / 20000
Forward pass
Cost =  0.0832211575849335
Backward pass
Updating weights...
Step : 19376 / 20000
Forward pass
Cost =  0.08322090110144358
Backward pass
Updating weights...
Step : 19377 / 20000
Forward pass
Cost =  0.08322064460941324
Backward pass
Updating weights...
Step : 19378 / 20000
Forward pass
Cost =  0.08322038810884
Backward pass
Updating weights...
Step : 19379 / 20000
Forward pass
Cost =  0.08322013159972137
Backward pass
Updating weights...
Step : 19380 / 20000
Forward pass
Cost =  0.08321987508205482
Backward pass
Updating weights...
Step : 19381 / 20000
Forward pass
Cost =  0.08321961855583777
Backward pass
Updating weights...
Step : 19382 / 20000
Forward pass
Cost =  0.08321936202106786
Backward pass
Updating weights...
Step : 19383 / 20000
Forward pass
Cost =  0.08321910547774243
Backward pass
Updating weights...
Step : 19384 / 20000
Forward pass
Cost =  0.08321884892585907
Backward pass
Updating weights...
Step : 19385 / 20000
For

Updating weights...
Step : 19667 / 20000
Forward pass
Cost =  0.08314589135608416
Backward pass
Updating weights...
Step : 19668 / 20000
Forward pass
Cost =  0.08314563227364831
Backward pass
Updating weights...
Step : 19669 / 20000
Forward pass
Cost =  0.08314537318195478
Backward pass
Updating weights...
Step : 19670 / 20000
Forward pass
Cost =  0.0831451140810012
Backward pass
Updating weights...
Step : 19671 / 20000
Forward pass
Cost =  0.08314485497078516
Backward pass
Updating weights...
Step : 19672 / 20000
Forward pass
Cost =  0.08314459585130425
Backward pass
Updating weights...
Step : 19673 / 20000
Forward pass
Cost =  0.08314433672255603
Backward pass
Updating weights...
Step : 19674 / 20000
Forward pass
Cost =  0.08314407758453822
Backward pass
Updating weights...
Step : 19675 / 20000
Forward pass
Cost =  0.08314381843724829
Backward pass
Updating weights...
Step : 19676 / 20000
Forward pass
Cost =  0.0831435592806839
Backward pass
Updating weights...
Step : 19677 / 20000
F

Cost =  0.0830787344457491
Backward pass
Updating weights...
Step : 19926 / 20000
Forward pass
Cost =  0.08307847289633127
Backward pass
Updating weights...
Step : 19927 / 20000
Forward pass
Cost =  0.08307821133704914
Backward pass
Updating weights...
Step : 19928 / 20000
Forward pass
Cost =  0.0830779497679004
Backward pass
Updating weights...
Step : 19929 / 20000
Forward pass
Cost =  0.08307768818888274
Backward pass
Updating weights...
Step : 19930 / 20000
Forward pass
Cost =  0.08307742659999387
Backward pass
Updating weights...
Step : 19931 / 20000
Forward pass
Cost =  0.0830771650012315
Backward pass
Updating weights...
Step : 19932 / 20000
Forward pass
Cost =  0.08307690339259333
Backward pass
Updating weights...
Step : 19933 / 20000
Forward pass
Cost =  0.08307664177407703
Backward pass
Updating weights...
Step : 19934 / 20000
Forward pass
Cost =  0.08307638014568033
Backward pass
Updating weights...
Step : 19935 / 20000
Forward pass
Cost =  0.08307611850740092
Backward pass
U

In [20]:
print('Test error :', cost.compute(net.predict(X_test), y_test))

Test error : 0.06481740048152486


# To-do :
- regularization term in the cost function
- dropout
- other optimization algorithms (maybe it's a bad idea to keep dW and db inside the layer's cache ?) : mini-batch GD, momentum, RMSProp, Adam, learning rate decay
- gradient check ?
- batch normalization
- softmax layer/activation function
- multi-task learning ? end-to-end learning ?
- CONVOLUTIONAL + POOLING LAYER
- blocks architecture : VGG, ResNet, Inception
